In [9]:
!pip install tensorflow==2.15

In [10]:
import tensorflow as tf
import pandas as pd

!pip install tf-explain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00


In [11]:
tf.__version__

'2.15.0'

In [12]:
from google.colab import auth
auth.authenticate_user()

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
!unzip '/content/drive/MyDrive/dataset.zip'

Streaming output truncated to the last 5000 lines.
  inflating: dataset/traindata/normal/Normal-3646.png  
  inflating: dataset/traindata/normal/Normal-3647.png  
  inflating: dataset/traindata/normal/Normal-3648.png  
  inflating: dataset/traindata/normal/Normal-3649.png  
  inflating: dataset/traindata/normal/Normal-365.png  
  inflating: dataset/traindata/normal/Normal-3650.png  
  inflating: dataset/traindata/normal/Normal-3651.png  
  inflating: dataset/traindata/normal/Normal-3652.png  
  inflating: dataset/traindata/normal/Normal-3653.png  
  inflating: dataset/traindata/normal/Normal-3654.png  
  inflating: dataset/traindata/normal/Normal-3655.png  
  inflating: dataset/traindata/normal/Normal-3656.png  
  inflating: dataset/traindata/normal/Normal-3657.png  
  inflating: dataset/traindata/normal/Normal-3658.png  
  inflating: dataset/traindata/normal/Normal-3659.png  
  inflating: dataset/traindata/normal/Normal-366.png  
  inflating: dataset/traindata/normal/Normal-3660.png  

In [15]:
imagesize = [299,299]
trainpath = '/content/dataset/traindata'

In [16]:
testpath = '/content/dataset/testdata'

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Activation,GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import Xception,preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [19]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [20]:
train =train_datagen.flow_from_directory(trainpath,target_size=(299,299),batch_size=32,class_mode='categorical')
test =test_datagen.flow_from_directory(testpath,target_size=(299,299),batch_size=32,class_mode='categorical')

Found 14679 images belonging to 4 classes.
Found 6665 images belonging to 4 classes.


In [21]:
xception =Xception(input_shape=imagesize+[3],weights= 'imagenet',include_top=False)


83683744/83683744 [==============================] - 0s 0us/step


In [22]:
for layer in xception.layers:
  layer.trainable = False

In [23]:
x = Flatten()(xception.output)


In [24]:
prediction= Dense(4,activation= 'softmax')(x)

In [25]:
model = Model(inputs=xception.input,outputs=prediction)

In [26]:

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 149, 149, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 149, 149, 32)         0         ['block1_conv1_bn[0][0]'] 

In [27]:
import tensorflow
callbacks = [
    tensorflow.keras.callbacks.EarlyStopping(patience=6)
]

In [28]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
r = model.fit_generator(train,validation_data=test,epochs=30,steps_per_epoch=len(train),validation_steps=len(test),callbacks=callbacks)

<ipython-input-28-fc55da4108eb>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(train,validation_data=test,epochs=30,steps_per_epoch=len(train),validation_steps=len(test),callbacks=callbacks)


Epoch 1/30
459/459 [==============================] - 443s 943ms/step - loss: 2.8611 - accuracy: 0.7394 - val_loss: 2.6365 - val_accuracy: 0.7754
Epoch 2/30
459/459 [==============================] - 468s 1s/step - loss: 2.7342 - accuracy: 0.7925 - val_loss: 3.3855 - val_accuracy: 0.7314
Epoch 3/30
459/459 [==============================] - 443s 966ms/step - loss: 2.8670 - accuracy: 0.7994 - val_loss: 6.2702 - val_accuracy: 0.7557
Epoch 4/30
459/459 [==============================] - 435s 948ms/step - loss: 2.8285 - accuracy: 0.8097 - val_loss: 4.5829 - val_accuracy: 0.7665
Epoch 5/30
459/459 [==============================] - 443s 965ms/step - loss: 2.7212 - accuracy: 0.8195 - val_loss: 3.2819 - val_accuracy: 0.7986
Epoch 6/30
459/459 [==============================] - 440s 959ms/step - loss: 2.7196 - accuracy: 0.8223 - val_loss: 5.0545 - val_accuracy: 0.7670
Epoch 7/30
459/459 [==============================] - 439s 956ms/step - loss: 2.8262 - accuracy: 0.8240 - val_loss: 4.8095 - va

In [37]:
model.save('CovidVision: Advanced COVID-19 Detection from Lung X-rays from Deep Learning.h5')
